**DATA SET**: Preprocessed

**Model Bert**: robert-large

**Kaggle Score**: 0.80765

**Fairness**: 3.98

**Best model was acquired in the 3rd Epoch:**

**Epoch 3**
* Training loss: 0.3006280800370537
* Validation loss: 0.5253806998637753
* |TL - VL| = 0,2247
* F1 Score (Macro): 0.80765



Installing transformers:

pip install transformers

**References**:


*   [RoBERTa: A Robustly Optimized BERT Pretraining Approach](https://arxiv.org/pdf/1810.04805.pdf?source=post_elevate_sequence_page---------------------------)
*   [RoBERTa](https://huggingface.co/transformers/model_doc/roberta.html) 

*   [RoBERTa For Sequence Classification](https://huggingface.co/transformers/model_doc/roberta.html#robertaforsequenceclassification)


*   [FineTuningRoBERTa](https://towardsdatascience.com/fine-tuning-bert-and-roberta-for-high-accuracy-text-classification-in-pytorch-c9e63cf64646)


In [ ]:
import pandas as pd
import numpy as np
import random


from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import tensorflow as tf

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm

from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup


In [ ]:
#Methods use to evaluate the model
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='macro')

def accuracy_per_class(preds, labels):
    label_dict_inverse = label_dict
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
#Setting GPU
if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
DATA_PATH = "../input/yogeshi/"
DATA_DEFI_PATH = "/kaggle/input/defiiainsatoulouse/"
LOGIT_OUTPUT_FILENAME = "logits_robertalarge_preprocessed_data.csv"

In [ ]:
preprocessed_train = pd.read_csv(DATA_PATH+"PreprocessedDataTrain.csv")
preprocessed_train= preprocessed_train.drop(['description'], axis = 1)
preprocessed_train = preprocessed_train.rename(columns = {'cleaned_Data':'description'}) 

train_label = pd.DataFrame(preprocessed_train['Category'], columns=['Category'])
label_dict = pd.read_csv(DATA_DEFI_PATH+"categories_string.csv")['0'].to_dict()

preprocessed_train['label_name'] = train_label.Category.replace(label_dict)
preprocessed_train['label'] =  train_label.Category

In [ ]:
seed_val = 23
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
tf.random.set_seed(seed_val)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(preprocessed_train.index.values, 
                                                  preprocessed_train.label.values, 
                                                  test_size=0.20, 
                                                  random_state=seed_val, 
                                                  stratify=preprocessed_train.label.values)

preprocessed_train['data_type'] = ['not_set']*preprocessed_train.shape[0]
preprocessed_train.loc[X_train, 'data_type'] = 'train'
preprocessed_train.loc[X_val, 'data_type'] = 'val'

df = preprocessed_train

In [ ]:
MAX_LENGTH = 60 
batch_size = 13   
epochs = 2

# AdamW Optimizer hyperparameters
learning_rate = 2e-5
eps = 1e-8 


tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
                                 
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    truncation=True,
    max_length=MAX_LENGTH, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    truncation=True,
    max_length=MAX_LENGTH, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:

model = RobertaForSequenceClassification.from_pretrained("roberta-large",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)


optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)
         

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
def evaluate(dataloader_val):

    model.eval()
    model.to(device)
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0] ,
                  'attention_mask': batch[1] ,
                  'labels':         batch[2] ,
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
#TRAINING AND SAVING THE MODEL BY EPOCH
lista_loss_train = []
lista_loss_val = []
lista_f1_val = []    
for epoch in tqdm(range(0, epochs+1)):
    
    model.train()
    model.to(device)
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
               
        inputs = {'input_ids':      batch[0] ,
                  'attention_mask': batch[1] ,
                  'labels':         batch[2] ,
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./finetuned_RoBERTaLarge_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    
    #Uncomment in case you want to generate the .csv file with all the ouputs
    #lista_loss_train.append(loss_train_avg)
    #lista_loss_val.append(val_loss)
    #lista_f1_val.append(val_f1)   
    
    #out = np.array([lista_loss_train,lista_loss_val,lista_f1_val])
    #out_df = pd.DataFrame(out.T, columns=['train_loss','val_loss','val_f1'])
    #out_df.to_csv('resultsByEpoch.csv',  index=False)
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Macro): {val_f1}')

**Epoch 1**
* Training loss: 0.6307210805501727
* Validation loss: 0.539798404291733
* |TL - VL| = 0,091
* F1 Score (Macro): 0.787909400212928

**Epoch 2**

* Training loss: 0.4297113419841447
* Validation loss: 0.5092155514573006
* |TL - VL| = 0,0795
* F1 Score (Macro): 0.8065729948741268


**Epoch 3** <=====  **Best Model Selected**

* Training loss: 0.3006280800370537
* Validation loss: 0.5253806998637753
* |TL - VL| = 0,2247
* F1 Score (Macro): 0.8134679670963686

# LOADING BEST 

In [ ]:
## LOADING BEST MODEL AND OUTPUTTING ACCURACY IN VALIDATION SET

BEST_MODEL_PATH = "./data_volume/finetuned_RoBERTaLarge_epoch_3.model"

model = RobertaForSequenceClassification.from_pretrained("roberta-large",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)


Class: pastor
Accuracy: 221/299

Class: model
Accuracy: 657/823

Class: yoga_teacher
Accuracy: 142/189

Class: teacher
Accuracy: 1122/1829

Class: personal_trainer
Accuracy: 125/161

Class: painter
Accuracy: 765/924

Class: journalist
Accuracy: 2012/2459

Class: interior_designer
Accuracy: 127/172

Class: surgeon
Accuracy: 980/1323

Class: accountant
Accuracy: 487/624

Class: dj
Accuracy: 126/166

Class: physician
Accuracy: 1708/2322

Class: comedian
Accuracy: 266/328

Class: software_engineer
Accuracy: 592/812

Class: nurse
Accuracy: 2173/2525

Class: poet
Accuracy: 667/859

Class: dentist
Accuracy: 1030/1090

Class: chiropractor
Accuracy: 169/281

Class: filmmaker
Accuracy: 670/825

Class: professor
Accuracy: 12940/14003

Class: photographer
Accuracy: 2603/2929

Class: rapper
Accuracy: 132/157

Class: psychologist
Accuracy: 1580/2078

Class: paralegal
Accuracy: 134/193

Class: architect
Accuracy: 820/1168

Class: composer
Accuracy: 578/679

Class: attorney
Accuracy: 3430/3764

Class: dietitian
Accuracy: 397/458

## LOAD FINAL TEST FILE


*   Storing logits in a csv file for  future use.

In [ ]:
preprocessed_test = pd.read_csv(DATA_PATH+"PreprocessedDataTest.csv") 
preprocessed_test= preprocessed_test.drop(['description'], axis = 1)
preprocessed_test = preprocessed_test.rename(columns = {'cleaned_Data':'description'}) 

In [ ]:
df  = preprocessed_test

encoded_data_test = tokenizer.batch_encode_plus(
    df.description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    truncation=True,
    max_length=MAX_LENGTH, 
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

dataset_test = TensorDataset(input_ids_test, attention_masks_test)
dataloader_test = DataLoader(dataset_test, batch_size=1)

model.eval()
model.to(device)
predictions = []

for batch in dataloader_test:
    batch = tuple(b.to(device) for b in batch)

    inputs = {'input_ids':      batch[0] ,
              'attention_mask': batch[1]             
             }
    
    with torch.no_grad(): 
        outputs = model(**inputs)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)



# Saving Logits in a .csv File
predictions_logits = np.asarray(predictions)
predictions_logits = predictions_logits.reshape(1,54300,28)
predictions_logits = np.squeeze(predictions_logits)
data_logits = pd.DataFrame(predictions_logits)  
data_logits.to_csv(LOGIT_OUTPUT_FILENAME, index=False)

#====================================================
# Generating output file for Kaggle submission
#predictions = np.concatenate(predictions, axis=0)
#preds_flat = np.argmax(predictions, axis=1).flatten()
#final_df = pd.DataFrame()
#final_df['Id'] = preprocessed_test.index
#final_df["Category"] = preds_flat
#file = final_df[['Id','Category']]
#file.to_csv("/kaggle/working/RobertaLargeModelYogeshiDB.csv", index=False)
#KAGGLE 0.80765
#====================================================